In [ ]:
!pip install scapy 
import sys
import scapy.all as scapy

In [ ]:
def process_packets(pcap_path):
    
    # First, there needs to be a counter created to sum the number of SYN and SYN-ACK requests
    syn_dict = dict()
    synack_dict = dict()

    for pkt in scapy.PcapReader(pcap_path):
        
        # Next, we want to ignore if not an TCP, Ethernet, or IP packet from the files
        if (not scapy.TCP in pkt) and (not scapy.Ether in pkt) and (not scapy.IP in pkt): 
            continue 
            
        # After examing the Wireshark pcap, it is clear that 0x2 is SYN packet and 0x10 is ACK    
        # If the packet is a SYN and an ACK, then we know it is probably a SYN-ACK
       
        
        if (scapy.TCP in pkt) and (pkt[scapy.TCP].flags & 0x2):
            if not (pkt[scapy.TCP].flags & 0x10):
                logged = pkt.sprintf('{IP:%IP.src%}{IPv6:%IPv6.src%}')
                if (logged not in syn_dict.keys()): 
                    syn_dict[logged]=0
                syn_dict[logged] =+ 1
                
        # Here is where we put the else if it is not a SYN entry  
         # This means it is added to the SYN-ACK count. 
            else:
                logged = pkt.sprintf('{IP:%IP.dst%}{IPv6:%IPv6.dst%}')
                if (logged not in synack_dict.keys()): 
                    synack_dict[logged]=0
                synack_dict[logged] =+ 1
    
        suspicious_ips = []

        for key in syn_dict:
        # Need to define an IP as suspicious
        #Defined as if there are 3x more SYNs than SYN-ACKs 
            if key in synack_dict and int(syn_dict[key]) /  int(synack_dict[key]) > 3:
                suspicious_ips.append(key)
        
        # Finally, it also doesn't make sense for there to be no SYN_ACK requests
        # This IPs are also going to be flagged as suspicious
            elif not (key in synack_dict):
                suspicious_ips.append(key)
    return suspicious_ips

In [ ]:
pcap_path = "part3/sample3.pcap" #THIS NEEDS TO BE CHANGED FOR WHERE YOU HAVE THE PCAP
suspicious_ips = process_packets(pcap_path)
for ip in suspicious_ips: print(ip)